In [1]:
# minimum edit distance the recursive (bad) way
calls = 0
function MED(s, t)
    global calls += 1
    cost = 0
    if s == ""
        return length(t) 
        # source is empty, so we need `t` insertion
    elseif t == ""
        return length(s) 
        # target is empty, so we need `s` deletion
    elseif t[end] == s[end]
        cost = 0
        # last character of source & target is equal
        # so we don't have to do anything
    else
        cost = 1
    end
#     println("$s $t") #uncomment to check how many times MED is called!
    total_cost = min(
        MED(t[1:end-1],s[1:end-1])+cost,
        MED(t[1:end-1],s)+1, #del
        MED(t,s[1:end-1])+1  #in
    )
    return total_cost
end
    
s,t = "Ciao", "Caio"

MED(s, t)
calls

481

In [2]:
# minimum edit distance the recursive (better) way with memory
calls = 0
function MED2(s, t)
    D = Dict()
    return MED!(D,s,t), D
end
function MED!(D, s, t)
    global calls += 1
    if s == ""
        return length(t) 
        # source is empty, so we need `t` insertion
    elseif t == ""
        return length(s) 
        # target is empty, so we need `s` deletion
    end
    
    cost = (t[end] == s[end]) ? 0 : 1

    # filter repetitive calls
    k1 = (s[1:end-1],t[1:end-1])
    if !( k1 in keys(D))
        D[k1] = MED!(D,k1...)
    end
    k2 = (s,t[1:end-1])
    if !( k2 in keys(D))
        D[k2] = MED!(D,k2...)
    end
    k3 = (s[1:end-1],t)
    if !( k3 in keys(D))
        D[k3] = MED!(D,k3...)
    end
    
    total_cost = min(
        D[k1]+cost,
        D[k2]+1, #del
        D[k3]+1  #in
    )
    return total_cost
end
    
ed, D = MED2("Ciao", "Caio")
calls

25

In [3]:
function iMED(s,t)
    n,m = length(s),length(t)
    D = zeros(Int,n+1,m+1)
    
    D[:,1] = 0:n # this is the cost we would have for insertion only
    D[1,:] = 0:m # this is the cost we would have for deletion only
    
    for i = 2:n+1, j = 2:m+1
       
        # check substition is needed
        cost = s[i-1] == t[j-1] ? 0 : 1
       
        D[i,j] = min(
                D[i-1,j] + 1,        # del
                D[i,j-1] + 1,        # ins
                D[i-1,j-1] + cost,   # subs / ok
                )
    end
    
    return D
end

s,t = "ciao", "ciriciao"

D = iMED(s,t)

5×9 Array{Int64,2}:
 0  1  2  3  4  5  6  7  8
 1  0  1  2  3  4  5  6  7
 2  1  0  1  2  3  4  5  6
 3  2  1  1  2  3  4  4  5
 4  3  2  2  2  3  4  5  4